In [ ]:
import requests
import pickle
import os

import pandas as pd 
import numpy as np
from tqdm import tqdm
import pubchempy as pcp

from urllib import request
from utils import *
import ast

import math

# 1. Download raw data from SABIO-RK
Downloading data from SABIO-RK can take a couple of hours.

In [ ]:
QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/kineticlawsExportTsv'
raw_data_save_path = "./sabiork_data_cache/sabio_origin_file/"
os.makedirs(raw_data_save_path,exist_ok=True)
file_list = os.listdir(raw_data_save_path) # Due to network instability, you can repeat several times to ensure that all data is downloaded
for i in range(1,10):
    for j in range(1,500):
        EC=f"{i}.{j}.*"
        if f"EcNumber{i}.{j}.pkl" in file_list:
            continue
        query_dict = {"ECNumber":'%s' %EC,}
        query_string = ' AND '.join(['%s:%s' % (k,v) for k,v in query_dict.items()])
        query = {'fields[]':['EntryID',
                            'Substrate', 'ECNumber', 'Organism',
                            'UniprotID', 
                            'EnzymeType', 
                            'PubMedID', 'SabioReactionID','KeggReactionID','Pathway', 'Buffer',
                            'pH',"temperature", 
                            'Smiles',
                            'Parameter' # kcat or Km value
                            ], 'q':query_string}
        request = requests.post(QUERY_URL, params = query)
        if request.status_code == 200:
            results = [i.split("\t") for i in request.text.split("\n")]
            if len(results)>2:
                print(f"EcNumber{i}.{j}.pkl")
                pickle.dump(results,open(raw_data_save_path+f"EcNumber{i}.{j}.pkl",'wb'))


# 2. Data cleaning
#### (a) Preprocess kcat data

In [ ]:
kcat_entry_dict = {}
for file in os.listdir(raw_data_save_path):
    results = pickle.load(open(raw_data_save_path+file,'rb'))
    for row in results[1:]:
        if len(row)<20: # Missing data
            continue
        EC = row[2] 
        if sum([1 if len(i)>=1 else 0 for i in EC.split(".")])!=4: # EC exception. Example: 1.1.1. Entry id 56619
            continue
        # Missing kcat value
        if row[14] == 'kcat' and row[16] == '': # Entry id 69933
            continue
        # Km-associated substrate is not in the substrate field above
        subs = row[1].split(';')
        if row[14] == 'Km' and row[15] not in subs: 
            continue
        entryId = row[0]
        if entryId not in kcat_entry_dict:
            kcat_entry_dict[entryId] = []
        kcat_entry_dict[entryId].append(row)
len(kcat_entry_dict)

In [ ]:
kcat_dataset = []
for entryId,results in kcat_entry_dict.items():
    EnzymeType = [row[14] for row in results]
    
    # Handling data with only one kcat value and at least one Km value.
    if EnzymeType.count("kcat")==1 and EnzymeType.count("Km")>0: 
        kcat_row = [row for row in results if row[14]=='kcat'][0] 
        subs = [row[15] for row in results if row[14]=='Km'] 
        for sub in set(subs):
            kcat_row_with_sub = list(kcat_row)
            kcat_row_with_sub[15]=sub
            kcat_dataset.append(kcat_row_with_sub)    
            
    # Handling data with no Km value, one kcat value, and only one substrate.
    elif EnzymeType.count("kcat")==1 and EnzymeType.count("Km")==0:
        sub = results[0][1].split(';')
        if len(sub) != 1:
            continue
        sub = sub[0]
        kcat_row = [row for row in results if row[14]=='kcat'][0] 
        kcat_row[15]=sub[0]
        kcat_dataset.append(kcat_row)    
    
    # Handling data with multiple kcat values and at least one Km value.
    elif EnzymeType.count("kcat")>1 and EnzymeType.count("Km")>0:
        web_results = get_page_from_sabiork(entry)
        results = entry_dict_multi_kcat[entry]
        subs = [row[2] for row in web_results if row[1] == 'Km']
        kcat_rows = [row for row in results if row[14] =='kcat' and row[19] == 's^(-1)']

        if len(set(subs))==1: # only one substrate for different Km values
            sub = subs[0]
            kcat_row = kcat_rows[0]
            kcat_row[16]=",".join([row[16] for row in kcat_rows])
            kcat_row[15]=sub
            kcat_dataset.append(kcat_row)
        else:               
            if 'kcat' in [row[0] for row in web_results]: # name cannot be mapped. e.g. Entry id 8246
                continue
            Km_tail = {row[0][-1]:row[2] for row in web_results if row[1] =='Km'} # Mapping substrate
            kcat_tail={}
            for row in web_results:
                if row[6] not in ['min^(-1)','s^(-1)']:
                    continue
                kcat = str(round(float(row[3])/60,6)) if row[6]=='min^(-1)' else row[3]
                kcat_tail[row[0][-1]]=kcat
            if sum([1 for tail in kcat_tail if list(Km_tail.keys()).count(tail) != 1]) != 0: # name cannot be mapped. e.g. Entry id 5316, 3247
                continue
            for kcat_row in kcat_rows:
                kcat_row_with_sub = list(kcat_row)

                tail = [t for t,v in kcat_tail.items() if round(float(v),3) == round(float(kcat_row_with_sub[16]),3)][0]
                sub = Km_tail[tail]
                Km_tail.pop(tail)
                kcat_tail.pop(tail)

                kcat_row_with_sub[15]=sub
                kcat_dataset.append(kcat_row_with_sub)
    else:
        continue
len(kcat_dataset)

#### (b) Preprocess Km data

In [ ]:
km_raw_dataset = []
filelist = os.listdir(raw_data_save_path)
need = []
for file in filelist:
    results = pickle.load(open(raw_data_save_path+file,'rb'))
    if len(results[0])==19:
        need.append(file.split('EcNumber')[1].split('.pkl')[0])
        continue
    for row in results[1:]:
        if len(row)<20:
            continue
        if row[14] != 'Km':
            continue
        if sum([1 if len(i)>=1 else 0 for i in row[2].split(".")])!=4: # e.g. Entry id 56619
            continue
        if row[14] == 'Km' and row[16] == '':  # e.g. Entry id 69733
            continue
        subs = row[1].split(';')
        if row[15] not in subs:
            continue
        if row[19] not in ['M','mg/ml'] :
            continue
        
        km_raw_dataset.append(row)
len(km_raw_dataset)

In [ ]:
mol_weight_dict = {}
for row in km_raw_dataset:
    if row[19] == 'mg/ml':
        mol_weight_dict[row[15]]=-1

for mol in tqdm(mol_weight_dict):
    molecular_weight = get_mol_weight(mol)
    if molecular_weight==-1:
        continue
    mol_weight_dict[mol]=molecular_weight

km_dataset = []
for raw_row in km_raw_dataset:
    row = list(raw_row)
    if row[19] == 'mg/ml':
        molecular_weight = mol_weight_dict[row[15]]
        if molecular_weight == -1:
            continue
        km_molar = float(row[16]) / (float(molecular_weight))
        row[16]=km_molar*1000
    elif row[19] == 'M':
        row[16] = float(row[16])*1000
    row[19]='mM'
    km_dataset.append(row)
len(km_dataset)

#### (c) Data cleaning

In [ ]:
EC_PATTERN = re.compile(r'^[1-9]\d*\.(?:0|[1-9]\d*)\.(?:0|[1-9]\d*)\.(?:0|[1-9]\d*)$')

def is_valid_ec(ec):
    if not isinstance(ec, str):
        return False
    ec = ec.strip()
    return bool(EC_PATTERN.match(ec))

kcat_dataset_clean=[row for row in kcat_dataset if row[4]!='' and len(row[4].split(" "))==1]
kcat_dataset_clean=[row for row in kcat_dataset is_valid_ec(row[2])]
kcat_dataset_clean=[row for row in kcat_dataset_clean if float(row[16])>0.00001 and float(row[16])<100000]

km_dataset_clean=[row for row in km_dataset if row[4]!='' and len(row[4].split(" "))==1]
km_dataset_clean=[row for row in km_dataset is_valid_ec(row[2])]
km_dataset_clean=[row for row in km_dataset_clean if float(row[16])>0.00001 and float(row[16])<100000]

In [ ]:
def get_origin_cluster_dict(dataset):
    field_loc = [2,3,4,6,15,8,10,11,12]    
    cluster_dict = {}
    for row in dataset:
        pair_name = ";;;".join([row[i] for i in field_loc])+';;;'+";;;".join(sorted(row[1].split(";")))+';;;'+";;;".join(sorted(row[13].split(";")))
        if row[5].startswith("mutant") and len(extract_mutations(row[5]))==0:
            continue
        if pair_name not in cluster_dict:
            cluster_dict[pair_name]=[]
        cluster_dict[pair_name].append(row)
            
    # Discard clusters containing only wild-type or only mutant entries.
    for v in list(cluster_dict.keys()):
        rows = cluster_dict[v]
        type = [1 if row[5].startswith('mutant') else 2 for row in rows]
        if type.count(1)>0 and type.count(2)>0:
            continue
        cluster_dict.pop(v)

    return cluster_dict

kcat_cluster_dict = get_origin_cluster_dict(kcat_dataset_clean)
km_cluster_dict = get_origin_cluster_dict(km_dataset_clean)
print(len(kcat_cluster_dict),len(km_cluster_dict))

# 3. Enzyme and substrate information retrieval

#### (a) Download SMILES

In [ ]:
Subs = [rows[0][15] for _,rows in kcat_cluster_dict.items()] + [rows[0][15] for _,rows in km_cluster_dict.items()]
Subs = list(set(Subs))
smiels_dict={}
for sub in tqdm(Subs):
    smiels_dict[sub]=get_comp(sub)
    if smiels_dict[sub]==-1:
        continue
    smiles = smiels_dict[sub].canonical_smiles
    if '.' in smiles:
        smiels_dict[sub]=-1
pickle.dump(smiels_dict,open("./sabiork_data_cache/smiles_dict.pkl",'wb'))

for cluster_name in list(kcat_cluster_dict.keys()):
    if smiels_dict[kcat_cluster_dict[cluster_name][0][15]] == -1:
        kcat_cluster_dict.pop(cluster_name)
for cluster_name in list(km_cluster_dict.keys()):
    if smiels_dict[km_cluster_dict[cluster_name][0][15]] == -1:
        km_cluster_dict.pop(cluster_name)
len(kcat_cluster_dict),len(km_cluster_dict)

#### (b) Download sequences and verify mutations

In [ ]:
error_aa = ['U','O','X','B','J','Z']
def check_aa(seq):
    for aa in seq:
        if aa in error_aa:
            return False
    return True

uniprotIds = list(set([rows[0][4] for _,rows in kcat_cluster_dict.items()] + [rows[0][4] for _,rows in km_cluster_dict.items()]))
UID_Seq_dict = {}
for id in tqdm(uniprotIds):
    url = "https://www.uniprot.org/uniprot/%s.fasta" % id
    try :
        data = request.urlopen(url)
        respdata = data.read().decode("utf-8").strip()
        seq = ''.join([i for i in respdata.split('\n')[1:]])
        if check_aa(seq):
            UID_Seq_dict[id] = seq
    except :
        print(id, "can not find from uniprot!")

In [ ]:
def check_mut_loc(cluster_dict):
    for pair_name in list(cluster_dict.keys()):
        rows = cluster_dict[pair_name]
        wt_rows = [row for row in rows if row[5].startswith("wildtype")]
        mut_rows = [row for row in rows if row[5].startswith("mutant")]
        mut_rows_right_mut_loc = []
        for row in mut_rows:
            mut_loc = extract_mutations(row[5])
            seq = IdSeq_dict[row[4]]
 
            for dev in range(-1,2,1):
                flag=True
                for mut in mut_loc:
                    loc = int(mut[1:-1])+dev
                    if loc>len(seq) or mut[0] != seq[loc]:
                        flag=False
                        break
                if flag:
                    mut_rows_right_mut_loc.append(row)
                    break
        if len(mut_rows_right_mut_loc)==0:
            cluster_dict.pop(pair_name)
    return cluster_dict

kcat_cluster_dict = check_mut_loc(kcat_cluster_dict)
km_cluster_dict = check_mut_loc(km_cluster_dict)
len(kcat_cluster_dict),len(km_cluster_dict)

# 4. Construct mutation effect pairs

#### (a) Deduplication
Deduplication of multiple wildtypes or multiple identical mutant data within a cluster requires manual intervention to ensure accuracy. Here, we exported the comments in the clusters to a text file for manual processing when controversies existed. For example, some clusters may contain modified enzymes (indicated by suffixes), which should be removed.

In [ ]:
def remove_cluster_multi_wt(cluster_dict):
    for pair_name in list(cluster_dict.keys()):
        rows = cluster_dict[pair_name]
        wt_rows = [row for row in rows if row[5].startswith("wild")]
        mut_rows = [row for row in rows if row[5].startswith("mutant")]
        mut_locs = [",".join(extract_mutations(row[5])) for row in mut_rows]
        
        if len(wt_rows)>1:
            print(wt_rows)
            K = ",".join([row[16] for row in wt_rows])
            wt_rows[0][16] = K
        wt_row = wt_rows[0]

        if len(set(mut_locs))!= len(mut_locs):
            grouped_mut_rows = [[row for row in mut_rows if ",".join(extract_mutations(row[5]))==mut] for mut in set(mut_locs)]
            mut_rows = []
            for sub_grouped_mut_rows in grouped_mut_rows:
                K = ",".join([row[16] for row in sub_grouped_mut_rows])
                sub_grouped_mut_rows[0][16] = K
                mut_rows.append(sub_grouped_mut_rows[0])    
        cluster_dict[pair_name] = [wt_row] + mut_rows
    return cluster_dict
    
# The cluster here is the cluster that has been manually confirmed
kcat_cluster_dict = remove_cluster_multi_wt(kcat_cluster_dict)
km_cluster_dict = remove_cluster_multi_wt(km_cluster_dict)
len(kcat_cluster_dict),len(km_cluster_dict)

#### (b) Create dataset and merge data

In [ ]:
def form_pairs(cluster_dict):
    delta_pairs = []
    for pair_name,rows in cluster_dict.items():
        wt_row = [row for row in rows if row[5].startswith("wild")][0]
        mut_rows = [row for row in rows if row[5].startswith("mutant")]
        uniprotId = wt_row[4]
        seq = IdSeq_dict[uniprotId]
        for mut_row in mut_rows:
            mut_loc = extract_mutations(mut_row[5])
            for dev in range(-1,2,1):
                flag=True
                for mut in mut_loc:
                    loc = int(mut[1:-1])+dev
                    if loc>=len(seq) or mut[0] != seq[loc]:
                        flag=False
                        break
                if flag: 
                    mut_loc_new = [mut[0]+str(int(mut[1:-1])+dev)+mut[-1] for mut in mut_loc]
                    delta_pairs.append([wt_row,mut_row,mut_loc_new])
                    break
    return delta_pairs

kcat_delta_pairs = form_pair(kcat_cluster_dict)
km_delta_pairs = form_pair(km_cluster_dict)
len(kcat_delta_pairs),len(km_delta_pairs)

In [ ]:
def create_df(delta_pair,target):
    EcNumber = []
    organism = []
    substrate = []
    keggReactionId = []
    UniprotId = []
    pubmedId = []
    temperature = []
    pH = []
    buffer = []

    sequence = []
    mutant = []
    wt_ks = []
    mut_ks = []
    delta_ks = []
    wt_entry = []
    mut_entry = []

    for wt_row,mut_row,mut_loc in delta_pair:
        wt_K,mut_K = [float(i) for i in str(wt_row[16]).split(',')],[float(i) for i in str(mut_row[16]).split(',')]
        wt_K = np.mean([math.log10(i) for i in wt_K])
        mut_K = np.mean([math.log10(i) for i in mut_K])
        
        EcNumber.append(wt_row[2])
        organism.append(wt_row[3].lower())
        substrate.append(wt_row[15].lower())
        keggReactionId.append(wt_row[8])
        UniprotId.append(wt_row[4])
        pubmedId.append(wt_row[6])
        
        temperature.append(str(float(wt_row[12])) if '-' not in wt_row[12] else '-')
        pH.append(str(float(wt_row[11])) if '-' not in wt_row[11] else '-')
        buffer.append(wt_row[10])

        sequence.append(IdSeq_dict[wt_row[4]])
        mutant.append(",".join(mut_loc))
        wt_ks.append(wt_K)
        mut_ks.append(mut_K)
        delta_ks.append(mut_K-wt_K)
        wt_entry.append(wt_row[0])
        mut_entry.append(mut_row[0])
    df = pd.DataFrame({
        'EcNumber':EcNumber,'Organism':organism,"Substrate":substrate,
        'KeggReactionId':keggReactionId,'UniprotId':UniprotId,'pubmedId':pubmedId,
        'Temperature':temperature,'pH':pH,'buffer':buffer,
        'sequence':sequence,'mutant':mutant,
        f'wt_{target}_log10':wt_ks,f'mut_{target}_log10':mut_ks,f"delta_{target}_log10":delta_ks
    })
    return df

delta_kcat_df = create_df(kcat_delta_pairs,'kcat')
delta_km_df = create_df(km_delta_pairs,'km')

In [ ]:
delta_kcat_df = delta_kcat_df.groupby(['EcNumber', 'Organism','Substrate','KeggReactionId','UniprotId',
                               'pubmedId','Temperature','pH','buffer','sequence','mutant',
                               'wt_kcat_log10','mut_kcat_log10'], as_index=False).agg({'delta_kcat_log10': 'mean'})
delta_km_df = delta_km_df.groupby(['EcNumber', 'Organism','Substrate','KeggReactionId','UniprotId',
                               'pubmedId','Temperature','pH','buffer','sequence','mutant',
                               'wt_km_log10','mut_km_log10'], as_index=False).agg({'delta_km_log10': 'mean'})

delta_kcat_df.to_csv("./sabiork_data_cache/sabiork_delta_kcat_df.csv",index=False)
delta_km_df.to_csv("./sabiork_data_cache/sabiork_delta_km_df.csv",index=False)